In [ ]:
from datetime import datetime, timezone
from dateutil import parser
from nba_api.stats.endpoints import  ScoreboardV2
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams

from agents.polymarket.gamma import GammaMarketClient as Gamma
import json

tag_slug = "nfl"
game_date = "2024-12-22"
gamma = Gamma()
querystring_params = {
    "limit": 1000,
    "active":True,
    "closed":False,
    "related_tags":True,
    "tag_slug":tag_slug
    # "slug": "nba-uta-por-2024-12-06"
    # "tag_id": "1,745,100639"
}
events = gamma.get_events(querystring_params=querystring_params)
print(f"len(events): {len(events)}")
with open("tmp.json", "w") as f:
    json.dump(events, f, indent=4)
filtered_events = []
for event in events:
    if "series" not in event:
        continue
    assert len(event["series"]) == 1
    if game_date in event["slug"]:
        filtered_events.append(event)
print(f"len(events): {len(filtered_events)}")
with open(f"assets/events_{tag_slug}_{game_date}.json", "w") as f:
    json.dump(filtered_events, f, indent=4)


In [ ]:
from datetime import datetime, timezone
from dateutil import parser
from nba_api.stats.endpoints import  ScoreboardV2
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams

from agents.polymarket.gamma import GammaMarketClient as Gamma
import json

slug = "will-bitcoin-hit-100k-today-12-20-24"
gamma = Gamma()
querystring_params = {
    "slug": slug
    # "tag_id": "1,745,100639"
}
events = gamma.get_events(querystring_params=querystring_params)
print(f"len(events): {len(events)}")
with open(f"assets/events_{slug}.json", "w") as f:
    json.dump(events, f, indent=4)

In [ ]:
# polymarket
from agents.polymarket.polymarket import Polymarket
import os
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds, BalanceAllowanceParams, AssetType
from dotenv import load_dotenv

host: str = "https://clob.polymarket.com"
chain_id: int = 137
load_dotenv()
key = os.getenv("POLYGON_WALLET_PRIVATE_KEY")
funder = os.getenv("FUNDER")
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=funder)  #
creds = client.create_or_derive_api_creds()
client.set_api_creds(creds)
print(creds)
balance = client.get_balance_allowance( params=BalanceAllowanceParams(asset_type=AssetType.COLLATERAL))["balance"]
balance = float(balance) / 1e6
# load csv
print(f"balance: {balance}")

In [ ]:
import time
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY, SELL
time_now = time.time()
token = "84605713417661453063334718202507496923189347137747272951220624704653962099528"
tick_size = float(client.get_tick_size(token))
print(f"get tick size cost {time.time() - time_now}")
print(f"tick_size: {tick_size}")
price = float(client.get_price(token, SELL)["price"])
price = min(price, 1-tick_size)
spread = float(client.get_spread(token)["spread"])
print(f"current price is {price}, spread is {spread}")
size = balance / price
print(f"Im buying {token} at {price} for {size} shares")
res = client.create_and_post_order(OrderArgs(
    price=price,
    size=size,
    side=BUY,
    token_id=token
))

# print(res)
# print(res)
print(f"price: {price} size: {size}")
print(f"time taken: {time.time() - time_now}")

In [ ]:
from tools.utils import get_team_token

tag_slug = "nfl"
game_date = "2024-12-08"
team_token = get_team_token(game_date=game_date, tag_slug=tag_slug)
print(f"team_token: {team_token}")


In [ ]:
234100 / 234100000

In [ ]:
import datetime
time_stamp = 1733548334
# to local time
local_time = datetime.datetime.fromtimestamp(time_stamp)
print(f"local_time: {local_time}")
# to utc time
utc_time = datetime.datetime.utcfromtimestamp(time_stamp)
print(f"utc_time: {utc_time}")
# to american ET time
et_time = datetime.datetime.fromtimestamp(time_stamp, tz=timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
print(f"et_time: {et_time}")

time_str = "2024-12-06 19:00:00"
stamp = datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S").timestamp()
print(f"stamp: {stamp}")

In [ ]:
from nba_api.stats.endpoints import  ScoreboardV2

from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import boxscoretraditionalv2


game_date = "2024-11-25"
board = ScoreboardV2(game_date=game_date)
data_sets = board.data_sets
for i, data_set in enumerate(data_sets):
    df = data_set.get_data_frame()
    if "GAME_ID" in df.columns and "HOME_TEAM_ID" in df.columns:
        game_ids = df["GAME_ID"].values
        for game_id in game_ids:
            if "339" not in str(game_id):
                continue
            # Query for games where the Celtics were playing
            print(f"game_id: {game_id}")

            # 获取比赛逐回合数据
            play_by_play = playbyplayv2.PlayByPlayV2(game_id=game_id)
            data = play_by_play.get_data_frames()[0]  # 返回DataFrame格式数据
            data.to_csv(f"play_by_play_{game_id}.csv", index=False)
            box_score =  boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
            for data_set in box_score.data_sets:
                df = data_set.get_data_frame()
                df.to_csv(f"box_score_{game_id}.csv", index=False)


            

In [ ]:
import requests
import json

# 定义 GraphQL 查询
query = """
{
  orderFilledEvents(
    where: { makerAssetId: "84472013145774660380603252188679283514256956886017890103299178370217717606440" }
    orderBy: timestamp
    orderDirection: desc
    first:500
  ) {
    id
    transactionHash
    timestamp
    maker {
      id
    }
    taker {
      id
    }
    makerAmountFilled
    takerAmountFilled
    fee
  }
}
"""

# 设置 GraphQL 端点 URL
url = "https://api.goldsky.com/api/public/project_cl6mb8i9h0003e201j6li0diw/subgraphs/polymarket-orderbook-resync/prod/gn"

# 发送请求
response = requests.post(url, json={'query': query})

# 获取并打印响应数据
data = response.json()
print(len(data))
with open("order_filled_event.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
# check flip
from tools.utils import check_flip, get_time_played
import time
time_str = "Q4 :13"
time_played = get_time_played(time_str)
print(f"time_played: {time_played}")
time_now = time.time()
flip_rate = check_flip(time_played, 5)
print(f"flip_rate: {flip_rate} time_taken: {time.time() - time_now}")

In [ ]:
from tools.utils import df
import logging
logging.basicConfig(level=logging.INFO)
gamids = df["GAME_ID"].values
print(len(gamids))
print(len(set(gamids)))

In [9]:
time_price = [
    (1.00011, 0.09999),
    (1.00011, 0.09998),
]
with open(f"assets/prices/t_t", "w") as f:
    for time_stamp, price in time_price:
        f.write(f"{time_stamp} {price:.6}\n")

In [ ]:
from datetime import datetime
import pytz

# UTC 时间
utc_time = "2024-12-22 18:00:00+00"

# 转换为 datetime 对象
utc_time_obj = datetime.fromisoformat(utc_time)

# 本地时区（自动根据系统时区）
local_time = utc_time_obj.astimezone()

print("本地时间:", local_time)
